# Load the CSV file

In this module, you will learn about:
- Creating nodes and relationships with data from a CSV file
- Assigning properties to nodes and relationships
- The importance of unique identifiers and how to create constraints

You will load a CSV file of "person" data into Person nodes in Neo4j. The CSV file contains the following fields:

- person_tmdbId
- bio
- born
- bornIn
- died
- person_imdb
- Id
- name
- person_poster
- person_url

Download the [persons.csv](https://data.neo4j.com/importing-cypher/persons.csv?_gl=1*15dcbv9*_ga*MTkzMzgxNTk1LjE3NTcyNTg0MzQ.*_ga_DZP8Z65KK4*czE3NjI2NDg5NzIkbzMyJGcxJHQxNzYyNjQ5NTA0JGoyMyRsMCRoMA..*_gcl_au*MjEzNTI4NjkxNy4xNzU3MjU4NDMzLjc4MDQ1OTczLjE3NTg0MTY3NjUuMTc1ODQxNjc2NA..*_ga_DL38Q8KGQC*czE3NjI2NDg5NzIkbzMyJGcxJHQxNzYyNjQ5NTA0JGoyMyRsMCRoMA..) file

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

import textwrap
import pandas as pd
from neo4j import GraphDatabase
from utils import execute_query


neo4j_uri = os.getenv("NEO4J_URI")
neo4j_user = os.getenv("NEO4J_USERNAME")
neo4j_pass = os.getenv("NEO4J_PASSWORD")
neo4j_db = os.getenv("NEO4J_DATABASE")


neo4j_driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_pass))


cypher = textwrap.dedent("""
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/persons.csv' AS row
RETURN row
""")

res = execute_query(neo4j_driver, cypher)

print(res)

# Create Person nodes

Run the Cypher statement to create the Person nodes:

```cypher
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/persons.csv' AS row
MERGE (p:Person {tmdbId: toInteger(row.person_tmdbId)})
SET
p.imdbId = toInteger(row.person_imdbId),
p.bornIn = row.bornIn,
p.name = row.name,
p.bio = row.bio,
p.poster = row.poster,
p.url = row.url,
p.born = row.born,
p.died = row.died
```

In [2]:
cypher = textwrap.dedent("""
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/persons.csv' AS row
MERGE (p:Person {tmdbId: toInteger(row.person_tmdbId)})
SET
p.imdbId = toInteger(row.person_imdbId),
p.bornIn = row.bornIn,
p.name = row.name,
p.bio = row.bio,
p.poster = row.poster,
p.url = row.url,
p.born = row.born,
p.died = row.died
""")

res = execute_query(neo4j_driver, cypher)

print(res)

Confirm the data is in the graph by returning the first 25 Person nodes:

In [3]:
cypher = textwrap.dedent("""
MATCH (p:Person) RETURN p LIMIT 25
""")

res = execute_query(neo4j_driver, cypher)

print(res)

# Unique IDs and Constraints



A Neo4j best practice is to use an ID as a unique property value for each node.

Unique IDs help ensure duplicate data is not created. When you load data from CSV files, you rely heavily upon the IDs specified in the file. If the IDs in your CSV file are not unique for the same entity (node), you could create duplicate data. You may also have problems loading the data and creating relationships between nodes.

You can add constraints to your database to stop the creation of nodes with duplicate IDs.

## Create a unique constraint

The syntax for creating a unique constraint on a property is:

```cypher
CREATE CONSTRAINT [constraint_name] [IF NOT EXISTS]
FOR (n:LabelName)
REQUIRE n.propertyName IS UNIQUE
```

The constraint is for a property on all nodes with a specified label.

The `constraint_name` is optional, but it is good practice to specify one. If you do not specify a constraint name, Neo4j will create one for you.

The `IF NOT EXISTS` clause is also optional - if not used Neo4j will generate an error if the constraint already exists.

## Person node constraint

The Person nodes you created should all have a unique tmbdId property.

You can create a constraint for the tmdbId property to ensure that all Person nodes have a unique `tmdbId` property value.

Review the following Cypher statement.
```cypher
CREATE CONSTRAINT Person_tmdbId IF NOT EXISTS
FOR (x:Person)
REQUIRE x.tmdbId IS UNIQUE
```

- The constraint name is Person_tmdbId.

- The optional clause IF NOT EXISTS is used - without which Neo4j would raise an error if the constraint exists.

- It applies to all nodes with the Person label.

- It requires the tmdbId property to be unique.


If you try to create a Person node with a duplicate tmdbId property value, Neo4j will raise an error.

```cypher
CREATE (p:Person {tmdbId: 3}) RETURN p
```

=>

```
22N80: Data exception - index entry conflict
Index entry conflict: Node(0) already exists with label `Label[0]` and property `PropertyKey[10]` = 3.
```

In [4]:
cypher = textwrap.dedent("""
CREATE CONSTRAINT Person_tmdbId
FOR (x:Person)
REQUIRE x.tmdbId IS UNIQUE
""")

res = execute_query(neo4j_driver, cypher)

### Show constraint

You can check that the constraint has been created by running the following Cypher statement:

```cypher
SHOW CONSTRAINTS
```

In [5]:
cypher = textwrap.dedent("""
SHOW CONSTRAINTS
""")

res = execute_query(neo4j_driver, cypher)

res

### Drop constaint
If you need to drop a constraint, use the following Cypher statement.

```cypher
DROP CONSTRAINT [constraint_name]
```

# Adding Movie Nodes

- Create nodes [movies.csv](https://data.neo4j.com/importing-cypher/movies.csv?_gl=1*1ivi1ei*_gcl_au*MjEzNTI4NjkxNy4xNzU3MjU4NDMzLjc4MDQ1OTczLjE3NTg0MTY3NjUuMTc1ODQxNjc2NA..*_ga*MTkzMzgxNTk1LjE3NTcyNTg0MzQ.*_ga_DL38Q8KGQC*czE3NjI2NjE3NDckbzMzJGcxJHQxNzYyNjYxODAwJGo3JGwwJGgw*_ga_DZP8Z65KK4*czE3NjI2NjE3NDckbzMzJGcxJHQxNzYyNjYxODAwJGo3JGwwJGgw)

- Create a constraint to ensure that there are no duplicate movies.

The file uses a comma (,) as a field delimiter and it contains headers and data for the following fields:

    - movieId
    - title
    - budget
    - countries
    - movie_imdbId
    - imdbRating
    - imdbVotes
    - languages
    - plot
    - movie_poster
    - released
    - revenue
    - runtime
    - movie_tmdbId
    - movie_url
    - year
    - genres


Set the following properties on the Movie nodes:

    - tmdbId
    - movieId
    - imdbId
    - released
    - title
    - year
    - plot
    - budget

In [6]:
df = pd.read_csv("https://data.neo4j.com/importing-cypher/movies.csv")
df.head()

,movieId,title,budget,countries,movie_imdbId,imdbRating,imdbVotes,languages,plot,movie_poster,released,revenue,runtime,movie_tmdbId,movie_url,year,genres
0,1,Toy Story,30000000.0,USA,114709,8.3,591836,English,A cowboy doll is profoundly threatened and jea...,https://image.tmdb.org/t/p/w440_and_h660_face/...,1995-11-22,373554033.0,81,862,https://themoviedb.org/movie/862,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,65000000.0,USA,113497,6.9,198355,English|French,When two kids find and play a magical board ga...,https://image.tmdb.org/t/p/w440_and_h660_face/...,1995-12-15,262797249.0,104,8844,https://themoviedb.org/movie/8844,1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men,NaN,USA,113228,6.6,18615,English,John and Max resolve to save their beloved bai...,https://image.tmdb.org/t/p/w440_and_h660_face/...,1995-12-22,NaN,101,15602,https://themoviedb.org/movie/15602,1995,Comedy|Romance
3,4,Waiting to Exhale,16000000.0,USA,114885,5.6,7210,English,"Based on Terry McMillan's novel, this film fol...",https://image.tmdb.org/t/p/w440_and_h660_face/...,1995-12-22,81452156.0,124,31357,https://themoviedb.org/movie/31357,1995,Comedy|Romance|Drama
4,5,Father of the Bride Part II,NaN,USA,113041,5.9,25938,English,"In this sequel, George Banks deals not only wi...",https://image.tmdb.org/t/p/w440_and_h660_face/...,1995-12-08,76578911.0,106,11862,https://themoviedb.org/movie/11862,1995,Comedy


In [7]:
cypher = textwrap.dedent("""
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/movies.csv' AS row
MERGE (p:Movie {tmdbId: toInteger(row.movie_tmdbId)})
SET
p.movieId = toInteger(row.movieId),
p.imdbId = toInteger(row.movie_imdbId),
p.released = row.released,
p.title = row.title,
p.year = row.year,
p.plot = row.plot,
p.budget = toFloat(row.budget)
""")

res = execute_query(neo4j_driver, cypher)

print(res)

In [8]:
cypher = textwrap.dedent("""
MATCH (m:Movie) RETURN m LIMIT 25
""")

res = execute_query(neo4j_driver, cypher)

print(res)

# Creating relationships

## ACTED_IN relationship

The [acted_in.csv](https://data.neo4j.com/importing-cypher/acted_in.csv) file contains the following data:

- `movieId` - the movieId property of the Movie node
- `person_tmdbId` - the tmbdId property of the Person node
- `role` - the role the person played in the movie

In [9]:
df = pd.read_csv("https://data.neo4j.com/importing-cypher/acted_in.csv")
df.head()

,movieId,person_tmdbId,role
0,1,12899,Slinky Dog (voice)
1,1,12898,Buzz Lightyear (voice)
2,1,31,Woody (voice)
3,1,7167,Mr. Potato Head (voice)
4,2,2157,Alan Parrish


In [10]:
cypher = textwrap.dedent("""
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/acted_in.csv' AS row
MATCH (p:Person {tmdbId: toInteger(row.person_tmdbId)})
MATCH (m:Movie {movieId: toInteger(row.movieId)})
MERGE (p)-[r:ACTED_IN]->(m)
SET r.role = row.role
""")

res = execute_query(neo4j_driver, cypher)

print(res)

In [14]:
cypher = textwrap.dedent("""
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie) RETURN p, r, m LIMIT 25
""")

res = execute_query(neo4j_driver, cypher)

print(res.keys())

['p', 'r', 'm']


## DIRECTED relationship

The [directed.csv](https://data.neo4j.com/importing-cypher/directed.csv) file contains just the movieId and person_tmdbId IDs. Unlike the ACTED_IN relationship, the DIRECTED relationship has no properties (e.g. role), so you will not need to use the SET clause.

In [19]:
df = pd.read_csv("https://data.neo4j.com/importing-cypher/directed.csv")
df.head()

,movieId,person_tmdbId
0,1,7879
1,2,4945
2,3,26502
3,4,2178
4,5,56106


In [20]:
cypher = textwrap.dedent("""
LOAD CSV WITH HEADERS
FROM 'https://data.neo4j.com/importing-cypher/directed.csv' AS row
MATCH (p:Person {tmdbId: toInteger(row.person_tmdbId)})
MATCH (m:Movie {movieId: toInteger(row.movieId)})
MERGE (p)-[r:DIRECTED]->(m);
""")

res = execute_query(neo4j_driver, cypher)

print(res)

In [21]:
cypher = textwrap.dedent("""
MATCH (p:Person)-[r:DIRECTED]->(m:Movie) RETURN p, r, m LIMIT 25
""")

res = execute_query(neo4j_driver, cypher)

print(res.keys())

['p', 'r', 'm']
